In [1]:
import os
os.chdir('../..')

In [2]:
import numpy as np
import pandas as pd

import ctgan
from ctgan import CTGANSynthesizer

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [3]:
dataset = 'Chicago'

In [4]:
df = pd.read_csv('../data/{}/data.csv'.format(dataset), index_col=False)

In [5]:
df.head()

,choice,travel_dow,trip_purpose,distance,hh_vehicles,hh_size,hh_bikes,hh_descr,hh_income,gender,age,license,education_level,work_status,departure_time
0,drive,7,HOME_OTHER,23.42579,2,2,0,2,6,1,66,1.0,6,FTE,9.333333
1,drive,7,OTHER,1.71259,2,2,0,2,6,1,66,1.0,6,FTE,12.083333
2,drive,7,HOME_OTHER,21.77887,2,2,0,2,6,1,66,1.0,6,FTE,15.500000
3,drive,7,SHOPPING,2.02603,2,2,0,2,6,1,66,1.0,6,FTE,17.500000
4,drive,7,SHOPPING,0.87691,2,2,0,2,6,1,66,1.0,6,FTE,18.250000


In [6]:
if dataset is 'Chicago':
    discrete_columns = [
        'choice',
        'travel_dow',
        'trip_purpose',
        'hh_vehicles',
        'hh_size',
        'hh_bikes',
        'hh_descr',
        'hh_income',
        'gender',
        'license',
        'education_level',
        'work_status'
    ]
elif dataset is 'LPMC':
    discrete_columns = [
        'travel_mode',
        'purpose',
        'fueltype',
        'faretype',
        'bus_scale',
        'survey_year',
        'travel_year',
        'travel_month',
        'travel_date',
        'day_of_week',
        'female',
        'driving_license',
        'car_ownership',
        'pt_n_interchanges',
        'cost_driving_con_charge'
    ]

In [7]:
output_folder = '../output/' + dataset + '/CTGAN/'

# Train

In [8]:
ctgan = CTGANSynthesizer(verbose=True, cuda=True, batch_size=200)

In [9]:
%%time
ctgan.fit(df, discrete_columns, epochs=10)

D:\Users\glede\anaconda3\envs\py37\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
D:\Users\glede\anaconda3\envs\py37\lib\site-packages\sklearn\mixture\_base.py:269: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)
D:\Users\glede\anaconda3\envs\py37\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
D:\Users\glede\anaconda3\envs\py37\lib\site-packages\sklearn\mixture\_base.py:269: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)
D:\Users

Epoch 1, Loss G: -0.0429,Loss D: -0.0021
Epoch 2, Loss G: -1.1990,Loss D:  0.0555
Epoch 3, Loss G: -1.2716,Loss D:  0.0381
Epoch 4, Loss G: -1.0611,Loss D: -0.0093
Epoch 5, Loss G: -1.1146,Loss D: -0.0498
Epoch 6, Loss G: -1.1835,Loss D:  0.0674
Epoch 7, Loss G: -0.5336,Loss D: -0.1670
Epoch 8, Loss G: -1.0655,Loss D: -0.0567
Epoch 9, Loss G: -1.2144,Loss D: -0.2221
Epoch 10, Loss G: -0.8776,Loss D:  0.0333
Wall time: 3min 16s


In [11]:
ctgan.save(output_folder + 'trained.pkl')